Part 1: Self-Study on ROC and AUC

· What is the ROC curve?
it shows the performance of a model by looking at the true positive rates against the false positives

· What does the AUC score represent?
This is the area under the curve that shows the performance on all thresholds and the higher the better

· How do ROC and AUC compare with other evaluation metrics like accuracy, precision, recall, and F1-score?
Use ROC for imbalanced datasets and when you need a threshold-independent metric.
And Precision, Recall, or F1-score when specific error types matter more.


Part 2: Applying Multiple Algorithms and Evaluating Metrics


1. Load and Prepare the Dataset

o Use the Titanic dataset (as in the class exercise).

o Preprocess the data as needed (handle missing values, encode categorical variables, etc.).

o Include the exercises we did in class, such as correlation and plotting a correlation matrix.

In [12]:
import pandas as pd
import numpy as np

df = pd.read_csv('titanic_clean.csv') 

df.replace('?', np.nan, inplace=True)

df.dropna(inplace=True)
print(df.columns)
print(df.dtypes)



Index(['Age', 'Cabin', 'Embarked', 'Fare', 'Name', 'Parch', 'PassengerId',
       'Pclass', 'Sex', 'SibSp', 'Survived', 'Ticket', 'Title', 'Family_Size'],
      dtype='object')
Age            float64
Cabin           object
Embarked        object
Fare           float64
Name            object
Parch            int64
PassengerId      int64
Pclass           int64
Sex             object
SibSp            int64
Survived       float64
Ticket          object
Title           object
Family_Size      int64
dtype: object


In [13]:
encoded = pd.get_dummies(df, columns=['Cabin', 'Embarked', 'Name','Sex','Ticket', 'Title'], drop_first=True)

corr_matrix = encoded.corr()
print(corr_matrix)

                       Age      Fare     Parch  PassengerId    Pclass  \
Age               1.000000 -0.071639 -0.254320     0.044056 -0.304142   
Fare             -0.071639  1.000000  0.383972     0.017243 -0.311713   
Parch            -0.254320  0.383972  1.000000    -0.063456  0.058366   
PassengerId       0.044056  0.017243 -0.063456     1.000000 -0.082977   
Pclass           -0.304142 -0.311713  0.058366    -0.082977  1.000000   
...                    ...       ...       ...          ...       ...   
Ticket_WE/P 5735  0.117032 -0.006894  0.145354     0.074638 -0.037186   
Title_Master     -0.393689 -0.027618  0.256872    -0.017377  0.186180   
Title_Miss       -0.291433  0.159140  0.151630    -0.101525  0.030447   
Title_Mr          0.291097 -0.122649 -0.185332     0.008827 -0.116235   
Title_Mrs         0.105762  0.008872 -0.024058     0.085657  0.039590   

                     SibSp  Survived  Family_Size  Cabin_A14  Cabin_A16  ...  \
Age              -0.141006 -0.220053    -0.

Train Multiple Classification Models

o In addition to the Random Forest classifier used in class, apply at least two more classification algorithms of your choice (e.g., Logistic Regression, Support Vector Machine, K-Nearest Neighbors, Gradient Boosting, etc.).

Evaluate Model Performance

o Compute the following metrics for each model:
§ Accuracy
§ Precision
§ Recall
§ F1-score
§ Confusion Matrix

o Display these metrics in an organized format (e.g., tables, visualizations).

Plot the ROC Curve and Compute AUC Score

o Generate the ROC curve for each model.

o Compute and report the AUC score.

o Compare the curves of different models in a single plot.

Part 3: Interpretation and Analysis


1. Compare Model Performance

o Compare the models based on evaluation metrics.

o Which model performed best based on accuracy, precision, recall, and F1-score?

Analyze the ROC Curve

o Examine the shape of the ROC curve for each model.

o Discuss what the AUC score reveals about the performance of each classifier.

o Explain in a markdown cell which model you would choose and why, based on the ROC-AUC analysis.